In [1]:
from enum import Enum
from typing import List
from collections import defaultdict

class Color(Enum):
    LIGHT_BLUE = 'L'
    ORANGE = 'O'
    BLUE = 'B'
    GREEN = 'G'
    MAGENTA = 'M'
    RED = 'R'
    YELLOW = 'Y'
    WHITE = 'W'
    BLACK = 'K'
    PURPLE = 'P'
    CYAN = 'C'

COLOR_ANSI = {
    Color.LIGHT_BLUE: 68,
    Color.ORANGE: 208,
    Color.BLUE: 27,
    Color.GREEN: 70,
    Color.MAGENTA: 13,
    Color.RED: 196,
    Color.YELLOW: 214,
    Color.WHITE: 244,
    Color.BLACK: 238,
    Color.PURPLE: 141,
    Color.CYAN: 45
}

class Marker(Enum):
    BLANK = ' '
    QUEEN = 'Q'
    NOT_QUEEN = '*'

class ContradictionException(Exception):
    pass

class IllegalMarkerChangeException(Exception):
    pass

class Cell:
    def __init__(self, row: int, col: int, color: Color, marker: Marker = Marker.BLANK):
        self.row = row
        self.col = col
        self.color = color
        self.marker = marker

# Format a string with ANSI color for pretty text output
def fmt(string: str, code: int, bg=False, reset=True):
  return u"\u001b[" + f"{48 if bg else 38};5;{code}m{string}" + ("\u001b[0m" if reset else "")

class Board:
    def __init__(self, grid: List[List[Cell]]):
        self.grid = grid
        self.M = len(self.grid)
        self.N = len(self.grid[0])

    def parse_from_text(file_path):
        inputs = [list(x.strip()) for x in open(file_path, 'r').readlines()]
        grid = []
        for i, line in enumerate(inputs):
            row = []
            for j, letter in enumerate(line):
                cell = Cell(i, j, Color(letter), Marker.BLANK)
                row.append(cell)
            grid.append(row)
        return Board(grid)
    
    def __str__(self):
        output = []
        for row in self.grid:
            line = []
            for cell in row:
                line.append(fmt(f"{cell.marker.value} ", COLOR_ANSI[cell.color], True))
            output.append(' '.join(line))
        return '\n'.join(output) + '\n'
    
    # Set marker, throw if it's an invalid change
    def set_marker(self, i, j, marker):
        curr_marker = self.grid[i][j].marker
        # Ignore NOPs
        if curr_marker == marker:
            return
        
        # Don't allow clearing cells
        if marker == Marker.BLANK:
            raise IllegalMarkerChangeException()
        
        # Allow placing into a blank cell
        if curr_marker == Marker.BLANK:
            self.grid[i][j].marker = marker
            return
        
        # Don't allow Q <-> X (usually a bad sign)
        raise IllegalMarkerChangeException()
    
    # Set cell to a queen, throw if invalid, handle X'ing row, col, diagonals
    def place_queen(self, i, j):
        # Set the cell to Queen
        self.set_marker(i, j, Marker.QUEEN)

        # Set other cells in the column to X
        for ni in range(0, self.M):
            if ni == i:
                continue
            self.set_marker(ni, j, Marker.NOT_QUEEN)

        # Set other cells in the row to X
        for nj in range(0, self.N):
            if nj == j:
                continue
            self.set_marker(i, nj, Marker.NOT_QUEEN)

        # Set the 4 diagonal cells to X
        for di, dj in [(-1, -1), (-1, 1), (1, -1), (1, 1)]:
            ni, nj = i + di, j + dj
            if 0 <= ni < self.M and 0 <= nj < self.N:
                self.set_marker(ni, nj, Marker.NOT_QUEEN)
        
        # Set other cells in the region to X
        for ni in range(self.M):
            for nj in range(self.N):
                if self.grid[ni][nj].color == self.grid[i][j].color:
                    if (ni, nj) == (i, j):
                        continue
                    self.set_marker(ni, nj, Marker.NOT_QUEEN)

    # TODO: Make sure this is called repeatedly
    def fill_certain_queens_strategy(self):
        color_freqs = defaultdict(lambda: defaultdict(set))
        row_freqs = defaultdict(lambda: defaultdict(set))
        col_freqs = defaultdict(lambda: defaultdict(set))

        for i in range(self.M):
            for j in range(self.N):
                cell = self.grid[i][j]
                color = cell.color
                marker = cell.marker
                color_freqs[color][marker].add((i, j))
                row_freqs[i][marker].add((i, j))
                col_freqs[j][marker].add((i, j))
        
        # For each row, col, and color region
        # If there is only one blank (and no queens), place a queen
        for zone_freqs in [row_freqs, col_freqs, color_freqs]:
            for freqs in zone_freqs.values():
                if len(freqs[Marker.BLANK]) == 1:
                    assert len(freqs[Marker.QUEEN]) == 0
                    bi, bj = list(freqs[Marker.BLANK])[0]
                    self.place_queen(bi, bj)

    # If all blanks for k colors are on just k lines, k queens have to be there
    # So X out all other blanks in the lines
    def fill_multi_owned_line_strat(self):
        # {color: {line_idx}}
        color_row_blanks = defaultdict(set)
        color_col_blanks = defaultdict(set)

        # {line_idx: {color}}
        row_color_blanks = defaultdict(set)
        col_color_blanks = defaultdict(set)

        for i in range(self.M):
            for j in range(self.N):
                cell = self.grid[i][j]
                color = cell.color
                marker = cell.marker
                if marker != Marker.BLANK:
                    continue
                color_row_blanks[color].add(i)
                color_col_blanks[color].add(j)
                row_color_blanks[i].add(color)
                col_color_blanks[j].add(color)

        # TODO: Raise the limit (needs to be much more efficient)
        max_num_colors = 5
        
        # TODO: Remove dupe calls. E.g. (purple, green) vs (green, purple)
        # E.g. {}, {}
        def claim_multiple_lines(prev_colors, prev_lines, is_row):
            # Cap number of color combos to explore
            if len(prev_colors) >= max_num_colors:
                return
            
            color_line_blanks = color_row_blanks if is_row else color_col_blanks

            for next_color, next_lines in color_line_blanks.items():
                if next_color in prev_colors:
                    continue

                new_colors = prev_colors | {next_color}
                new_lines = prev_lines | next_lines

                # Too many lines to fill
                if len(new_lines) > max_num_colors:
                    continue

                # k colors span k lines
                if len(new_lines) == len(new_colors):
                    for line in new_lines:
                        if is_row:
                            for j in range(self.N):
                                # So, set all *other* blanks in the row(s) to X
                                if self.grid[line][j].color not in new_colors:
                                    self.set_marker(line, j, Marker.NOT_QUEEN)
                                    # TODO: Remove color from consideration in deeper levels?
                        else:
                            for i in range(self.M):
                                # So, set all *other* blanks in the col(s) to X
                                if self.grid[i][line].color not in new_colors:
                                    self.set_marker(i, line, Marker.NOT_QUEEN)
                                    # TODO: Remove color from consideration in deeper levels?

                claim_multiple_lines(new_colors, new_lines, is_row)
        
        claim_multiple_lines(set(), set(), False)
        claim_multiple_lines(set(), set(), True)

        # TODO: Make more efficient by starting with smaller colors 
        # and only using overlapping ones
    
    # TODO: Strategy to x out border around 2 remaining blanks of a color.
    #              xx                    x
    # E.g. RR  ->  RR       or R R  ->  R R
    #              xx                    x
    # TODO: Also make work for like L shape that also can't have queen next to it

    def is_solved(self):
        return self.is_filled() and self.is_valid()

    # Check if a board is totally filled
    def is_filled(self):
        num_blanks = 0

        for i in range(self.M):
            for j in range(self.N):
                cell = self.grid[i][j]
                marker = cell.marker
                if marker == Marker.BLANK:
                    return False
                
        return True

    # Check that a board is valid
    def is_valid(self):
        color_freqs = defaultdict(lambda: defaultdict(set))
        row_freqs = defaultdict(lambda: defaultdict(set))
        col_freqs = defaultdict(lambda: defaultdict(set))
        queen_locs = set()

        for i in range(self.M):
            for j in range(self.N):
                cell = self.grid[i][j]
                color = cell.color
                marker = cell.marker
                color_freqs[color][marker].add((i, j))
                row_freqs[i][marker].add((i, j))
                col_freqs[j][marker].add((i, j))
                if marker == Marker.QUEEN:
                    queen_locs.add((i, j))
        
        # For each row, col, and color region
        # Ensure there is 1 queen OR some blanks left
        for zone_freqs in [row_freqs, col_freqs, color_freqs]:
            for freqs in zone_freqs.values():
                # If no queens and no blanks, bad news
                if len(freqs[Marker.QUEEN]) == 0 and len(freqs[Marker.BLANK]) == 0:
                    return False
                # If multiple queens, bad news
                if len(freqs[Marker.QUEEN]) > 1:
                    return False
        
        # Check that queens aren't diagonal from each other
        for i, j in queen_locs:
            for di, dj in [(-1, -1), (-1, 1), (1, -1), (1, 1)]:
                ni, nj = i + di, j + dj
                if 0 <= ni < self.M and 0 <= nj < self.N:
                    if self.grid[ni][nj].marker == Marker.QUEEN:
                        return False
        
        return True

    def same_as(self, other):
        if self.M != other.M:
            return False
        if self.N != other.N:
            return False
        for i in range(self.M):
            for j in range(self.N):
                cell = self.grid[i][j]
                other_cell = other.grid[i][j]
                if cell.color != other_cell.color:
                    return False
                if cell.marker != other_cell.marker:
                    return False
        return True

    def copy(self):
        grid = []
        for i in range(self.M):
            row = []
            for j in range(self.N):
                other_cell = self.grid[i][j]
                cell = Cell(i, j, other_cell.color, other_cell.marker)
                row.append(cell)
            grid.append(row)
        return Board(grid)
    

def pick_a_blank(board):
    for i in range(board.M):
        for j in range(board.N):
            color_blanks = defaultdict(set)
            row_blanks = defaultdict(set)
            col_blanks = defaultdict(set)

            for i in range(board.M):
                for j in range(board.N):
                    cell = board.grid[i][j]
                    color = cell.color
                    marker = cell.marker
                    if marker == Marker.BLANK:
                        color_blanks[color].add((i, j))
                        row_blanks[i].add((i, j))
                        col_blanks[j].add((i, j))
    
    best_zone = None
    for zone_blanks in [color_blanks, row_blanks, col_blanks]:
        for blanks in zone_blanks.values():
            if not best_zone or len(blanks) < len(best_zone):
                best_zone = blanks
    
    blanks = sorted(best_zone)
    return blanks[0]

def solve(board):
    # Apply strategies until the board doesn't change
    old_board = None
    while not old_board or not board.same_as(old_board):
        old_board = board.copy()
        board.fill_multi_owned_line_strat()
        board.fill_certain_queens_strategy()

    if board.is_solved():
        return board

    if board.is_filled():
        raise ContradictionException('Board completed but invalid')
    
    # Try placing a queen in a blank spot
    i, j = pick_a_blank(board)
    try:
        board.place_queen(i, j)
        solution = solve(board)
    # If that didn't end up working out, it can't be a queen
    except (ContradictionException, IllegalMarkerChangeException) as e:
        board = old_board
        board.set_marker(i, j, Marker.NOT_QUEEN)
        solution = solve(board)

    return solution

In [2]:
# This cell has generic image parsing helpers

from PIL import Image, ImageDraw
import numpy as np
import cv2
import statistics
import math

# Check if inner box is contained in the outer box
def is_contained(inner, outer):
    x1, y1, x2, y2 = inner
    ox1, oy1, ox2, oy2 = outer
    return x1 >= ox1 and y1 >= oy1 and x2 <= ox2 and y2 <= oy2

# Check if two boxes overlap
def are_overlapping(rect_a, rect_b):
    ax1, ay1, ax2, ay2 = rect_a
    bx1, by1, bx2, by2 = rect_b
    return (ax1 <= bx1 <= ax2 and ay1 <= by1 <= ay2) or (ax1 <= bx2 <= ax2 and ay1 <= by2 <= ay2)

# Remove boxes that are nested within each other
def remove_nested_boxes(boxes):
    filtered = []
    for i, box in enumerate(boxes):
        if not any(i != j and is_contained(box, other) for j, other in enumerate(boxes)):
            filtered.append(box)
    return filtered

# Merge together overlapping boxes
# TODO: Not very efficient
def merge_overlapping_boxes(boxes):
    while True:
        new_boxes = set()
        overlapping_boxes = set()
        # Iterate over first boxes
        for i, box_a in enumerate(boxes):
            # Iterate over second boxes
            for j in range(i + 1, len(boxes)):
                box_b = boxes[j]
                # Check if the boxes overlap
                if are_overlapping(box_a, box_b):
                    overlapping_boxes.add(box_a)
                    overlapping_boxes.add(box_b)
                    # Add the merged box to our new list
                    new_boxes.add((min(box_a[0], box_b[0]), min(box_a[1], box_b[1]), max(box_a[2], box_b[2]), max(box_a[3], box_b[3])))
                    break
            # Add box A to our new list, if it wasn't merged
            if box_a not in overlapping_boxes:
                new_boxes.add(tuple(box_a))
        boxes = list(new_boxes)
        # Stop trying to merge if we didn't find any
        if not overlapping_boxes:
            break
    return boxes

# Find bounding boxes around contours made of the given colors
# Optional: Filter by size, draw on a debug image, merge overlapping boxes, remove nested boxes
def find_boxes(image_np, target_colors=[], min_size=0, remove_nested=False, max_size=float("inf"), draw=None, box_color='magenta', tolerance=2, merge_overlapping=False, width=1):
    # In the image, find pixels close enough to any of the target colors
    mask = None
    for target_color in target_colors:
        target_color = np.array(target_color)
        dist = np.linalg.norm(image_np - target_color, axis=2)
        new_mask = np.array(dist < tolerance).astype(np.uint8) * 255
        if mask is None:
            mask = new_mask
        else:
            mask |= new_mask

    # Create contours around the matching pixels
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Compute bounding boxes for the contours. Filter boxes by size.
    boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if max_size >= w >= min_size and max_size >= h >= min_size:
            boxes.append((x, y, x + w, y + h))
    
    # Remove nested boxes
    if remove_nested:
        boxes = remove_nested_boxes(boxes)

    # Merge overlapping boxes
    if merge_overlapping:
        boxes = merge_overlapping_boxes(boxes)

    # Draw the boxes
    if draw is not None:
        for box in boxes:
            draw.rectangle(box, outline=box_color, width=width)

    return boxes

# Draw boxes on the debug image
def draw_boxes(boxes, box_color, draw, width=1):
    if draw is None:
        return
    for box in boxes:
        draw.rectangle(box, outline=box_color, width=width)

# Convert an image greyscale and normalize to [0, 1]
def normalize_greyscale(img):
    return np.array(img.convert("L")) / 255.0

# Load a mask in normalized greyscale
def load_mask(path):
    return normalize_greyscale(Image.open(path))

# Load multiple masks, in normalized greyscale
def load_masks(paths):
    return [load_mask(path) for path in paths]

# Find distance between the image and the mask (resize the mask)
# TODO: rename
def distance_to_mask(img_np, mask_np):
    # Scale mask to image size
    mask_resized = Image.fromarray((mask_np * 255).astype(np.uint8)).resize(img_np.shape[::-1], Image.Resampling.BILINEAR)
    # Convert to normalized array
    mask_resized_np = np.array(mask_resized) / 255.0
    # Calculate the L2 distance between the mask and image
    return np.linalg.norm(img_np - mask_resized_np)

# Return the label of the mask that matches the image best
# TODO: rename
def classify_symbol(img, masks):
    img_np = normalize_greyscale(img)
    scores = {label: float("inf") for label in masks}
    for label, mask_nps in masks.items():
        for mask_np in mask_nps:
            score = distance_to_mask(img_np, mask_np)
            scores[label] = min(score, scores[label])
    closest = min(scores, key=scores.get)
    return closest, scores

# Parse a screenshot from LinkedIn
def parse_linkedin_image(image_path, debug=False):
    # TODO: Make debug line width thicker for larger images

    # Parameters for find_boxes, for all colored cells
    # TODO: Rename
    target_colors = {
        # Color: box_color, min_size, remove_nested, max_size, tolerance, merge_overlapping
        '': ([[163, 210, 216], [255, 201, 146], [150, 190, 255], [179, 223, 160], [223, 160, 191], [255, 123, 96], 
              [230, 243, 136], [223, 223, 223], [185, 178, 158], [187, 163, 226], [98, 239, 234]], 
              'white', 90, False, 180, 2, False)
    }

    # Colors used to classify cells
    cell_colors = {
        Color.LIGHT_BLUE: [163, 210, 216],
        Color.ORANGE: [255, 201, 146],
        Color.BLUE: [150, 190, 255],
        Color.GREEN: [179, 223, 160],
        Color.MAGENTA: [223, 160, 191],
        Color.RED: [255, 123, 96],
        Color.YELLOW: [230, 243, 136],
        Color.WHITE: [223, 223, 223],
        Color.BLACK: [185, 178, 158],
        Color.PURPLE: [187, 163, 226],
        Color.CYAN: [98, 239, 234],
    }

    # Colors used as cell edges
    cell_borders = {
        Color.LIGHT_BLUE: 'skyblue',
        Color.ORANGE: 'orange',
        Color.BLUE: 'blue',
        Color.GREEN: 'green',
        Color.MAGENTA: 'magenta',
        Color.RED: 'red',
        Color.YELLOW: 'yellow',
        Color.WHITE: 'lightgrey',
        Color.BLACK: 'darkgray',
        Color.PURPLE: 'purple',
        Color.CYAN: 'cyan'
    }

    # Load game image
    image = Image.open(image_path).convert('RGB')
    image_np = np.array(image)

    # Create a copy of the image to draw debug info on
    image_draw = draw = None
    if debug:
        image_draw = image.copy()
        draw = ImageDraw.Draw(image_draw)

    # Find the board bounding box
    board_box = find_boxes(image_np, [[0, 0, 0]], 150, False, 1250, None, None, 5, False)
    x_min, y_min, x_max, y_max = board_box[0]
    if debug: print(f"board box: {board_box[0]}")
    x_min, y_min, x_max, y_max = x_min + 13, y_min + 13, x_max - 13, y_max - 13

    # Find the board dimensions
    board_width, board_height = x_max - x_min, y_max - y_min
    if debug: print(f"board size: {board_width} x {board_height}")

    # Draw the border
    if debug: draw.rectangle(board_box[0], outline="grey", width=4)

    # Create a crop of just the board
    board_img_np = image_np[y_min:y_max, x_min:x_max]

    # Find starter cells. Only search within the board.
    starter_boxes = []
    for name, (target_colors, box_color, min_size, remove_nested, max_size, tolerance, merge_overlapping) in target_colors.items():
        boxes = find_boxes(board_img_np, target_colors, min_size, remove_nested, max_size, None, None, tolerance, merge_overlapping)
        boxes = [(x1 + x_min, y1 + y_min, x2 + x_min, y2 + y_min) for x1, y1, x2, y2 in boxes]
        draw_boxes(boxes, box_color, draw, width=1)
        starter_boxes.extend(boxes)

    # Find the width and height of the cells
    cell_width = max([box[2] - box[0] for box in starter_boxes])
    cell_height = max([box[3] - box[1] for box in starter_boxes])
    if debug: print(f"cell size: {cell_width:.1f} x {cell_height:.1f}")

    # Guess the number of rows and columns
    num_rows, num_cols = int(board_width / cell_width), int(board_height / cell_height)
    if debug: print(f"board dims: {num_rows} x {num_cols}")

    # Calculate the gap between each cell
    cell_gap = (board_width - (cell_width * num_cols)) / (num_cols - 1)
    if debug: print(f"cell gap: {cell_gap:.1f}")

    # Populate cells by sampling a pixel
    # TODO: Turn into a helper function, since this is used for both LinkedIn and the Tango App. If you do, careful with how pixel is calculated
    cells = [[None for _ in range(num_cols)] for _ in range(num_rows)]
    for r in range(num_rows):
        for c in range(num_cols):
            # Sample a pixel near the middle of the cell
            x = int(x_min + (cell_width + cell_gap) * c + 0.25 * cell_width)
            y = int(y_min + (cell_height + cell_gap) * r + 0.25 * cell_height)
            pixel = image_np[y, x, :]
            # Find which cell color this pixel is closest to
            distances = {name: np.linalg.norm(pixel - color) for name, color in cell_colors.items()}
            closest = min(distances, key=distances.get)
            cells[r][c] = closest
            # Draw the cell and type on the debug image
            if cell_borders[closest] is not None:
                x1 = x_min + (cell_width + cell_gap) * c
                y1 = y_min + (cell_height + cell_gap) * r
                x2, y2 = x1 + cell_width, y1 + cell_height
                rect = list(map(round, [x1, y1, x2, y2]))
                if debug: draw.rectangle(rect, outline=cell_borders[closest], width=6)

    # Show debug image
    if debug: image_draw.show()

    # Initialize puzzle
    board = Board([[Cell(i, j, color, Marker.BLANK) for j, color in enumerate(row)] for i, row in enumerate(cells)])
    
    return board

In [3]:
import time

# text_file = 'text_inputs/2025-05-04.txt'
# text_file = 'text_inputs/2025-05-05.txt'
# text_file = 'text_inputs/2025-05-07.txt'
# text_file = 'text_inputs/2025-05-08.txt'
# text_file = 'text_inputs/2025-05-28.txt'
# text_file = 'text_inputs/2025-06-29.txt'
text_file = 'text_inputs/2025-07-03.txt'

start = time.time()
board = Board.parse_from_text(text_file)
print(f"Parsed text in {time.time() - start:0.3f} seconds")

start = time.time()
solution = solve(board.copy())
print(f"Solved in {time.time() - start:0.3f} seconds")
print(board)
print(solution)

Parsed text in 0.001 seconds
Solved in 0.176 seconds
                          
                          
                          
                          
                          
                          
                          
                          
                          

*  *  *  *  *  *  Q  *  * 
*  *  Q  *  *  *  *  *  * 
*  *  *  *  Q  *  *  *  * 
Q  *  *  *  *  *  *  *  * 
*  *  *  Q  *  *  *  *  * 
*  *  *  *  *  *  *  *  Q 
*  Q  *  *  *  *  *  *  * 
*  *  *  *  *  *  *  Q  * 
*  *  *  *  *  Q  *  *  * 



In [4]:
# img_path = 'image_inputs/2025-05-04.png'
# img_path = 'image_inputs/2025-05-05.png'
# img_path = 'image_inputs/2025-05-07.png'
# img_path = 'image_inputs/2025-05-08.png'
# img_path = 'image_inputs/2025-05-28.png'
# img_path = 'image_inputs/2025-06-29.png'
# img_path = 'image_inputs/2025-07-03.png'
img_path = 'screenshots/2025-05-28-input.png'

start = time.time()
board = parse_linkedin_image(img_path, debug=False)
print(f"Parsed image in {time.time() - start:0.3f} seconds")

start = time.time()
solution = solve(board.copy())
print(f"Solved in {time.time() - start:0.3f} seconds")
print()
print(board)
print(solution)

Parsed image in 1.013 seconds
Solved in 0.087 seconds

                                
                                
                                
                                
                                
                                
                                
                                
                                
                                
                                

*  *  *  *  *  *  *  *  Q  *  * 
*  *  *  *  *  Q  *  *  *  *  * 
*  Q  *  *  *  *  *  *  *  *  * 
*  *  *  *  *  *  Q  *  *  *  * 
*  *  *  Q  *  *  *  *  *  *  * 
*  *  *  *  *  *  *  Q  *  *  * 
*  *  *  *  *  *  *  *  *  *  Q 
*  *  *  *  Q  *  *  *  *  *  * 
*  *  *  *  *  *  *  *  *  Q  * 
Q  *  *  *  *  *  *  *  *  *  * 
*  *  Q  *  *  *  *  *  *  *  * 

